In [1]:
import json
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model
from transformers import TrainingArguments, Trainer
import torch

# -----------------------------
# 1. Cargar dataset
# -----------------------------
dataset = load_dataset("json", data_files="tutor_dataset2.jsonl")

# -----------------------------
# 2. Cargar modelo base (CPU optimizado)
# -----------------------------
model_name = "microsoft/Phi-3.5-mini-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Cargar modelo en CPU con dtype float32 (sin cuantización)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float32,  # Usar float32 para CPU
    low_cpu_mem_usage=True,     # Optimizar uso de memoria
    device_map="cpu"            # Forzar CPU
)

# -----------------------------
# 3. Configurar LoRA (ajustado para Phi-3.5)
# -----------------------------
lora_config = LoraConfig(
    r=8,                        # Phi-3.5 mini es pequeño, podemos usar r=8
    lora_alpha=16,              
    target_modules=["qkv_proj", "o_proj"],  # Módulos correctos para Phi-3.5
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
print(f"Parámetros entrenables: {model.print_trainable_parameters()}")

# -----------------------------
# 4. Preprocesar el dataset
# -----------------------------
def format_instruction(example):
    # Formato específico para Phi-3.5
    prompt = f"<|user|>\n{example['instruction']}<|end|>\n<|assistant|>\n"
    full_text = prompt + example["response"] + "<|end|>"
    
    # Tokenizar con longitud máxima
    return tokenizer(
        full_text, 
        truncation=True, 
        max_length=512,
        padding="max_length"
    )

tokenized = dataset.map(
    format_instruction, 
    remove_columns=dataset["train"].column_names
)

# -----------------------------
# 5. Entrenamiento (configuración para CPU)
# -----------------------------
training_args = TrainingArguments(
    output_dir="./lora-tutor3",
    per_device_train_batch_size=2,      # Phi-3.5 es más pequeño, podemos usar batch_size=2
    gradient_accumulation_steps=8,      
    logging_steps=10,
    num_train_epochs=3,                 # 3 épocas es razonable para Phi-3.5
    fp16=False,                         
    save_steps=100,
    save_total_limit=2,                 
    learning_rate=2e-4,
    warmup_steps=50,
    lr_scheduler_type="cosine",
    dataloader_num_workers=0,           
    optim="adamw_torch",                
    max_grad_norm=1.0,
    report_to="none",                   
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

print("Iniciando entrenamiento en CPU (esto puede ser LENTO)...")
trainer.train()

# -----------------------------
# 6. Guardar adaptadores LoRA
# -----------------------------
model.save_pretrained("./lora-tutor3")
tokenizer.save_pretrained("./lora-tutor3")
print("✅ Entrenamiento completado. Adaptadores guardados en ./lora-tutor3")

# -----------------------------
# 7. Ejemplo de inferencia (opcional)
# -----------------------------
print("\n--- Prueba de inferencia ---")
model.eval()
test_prompt = "<|user|>\n¿Cómo crear una lista en Java?<|end|>\n<|assistant|>\n"
inputs = tokenizer(test_prompt, return_tensors="pt")

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        temperature=0.7,
        do_sample=True
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

2025-12-11 09:57:31.574513: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-11 09:57:31.612042: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-11 09:57:34.787543: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


Generating train split: 0 examples [00:00, ? examples/s]

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 4,718,592 || all params: 3,825,798,144 || trainable%: 0.1233
Parámetros entrenables: None


...

The model is already on multiple devices. Skipping the move to device specified in `args`.


Iniciando entrenamiento en CPU (esto puede ser LENTO)...


/home/yisus/.config/jupyterlab-desktop/jlab_server/lib/python3.12/site-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,2.367800
20,1.937000
30,1.415700
40,1.125900
50,0.930000
60,0.841300
70,0.806600
80,0.770400
90,0.802000


✅ Entrenamiento completado. Adaptadores guardados en ./lora-tutor3

--- Prueba de inferencia ---
¿Cómo crear una lista en Java? Las listas (ArrayList) son colecciones dinámicas que pueden expandirse de forma automatica.

```java
import java.util.ArrayList;

public class Listas {
    public static void main(String[] args) {
        ArrayList<String> nombres = new ArrayList<>();
        nombres.add("Alice");
        System.out.println(nombres.size()); // Salida: 1
    }
}```


In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

# -----------------------------
# 1. Configuración
# -----------------------------
model_name = "microsoft/Phi-3.5-mini-instruct" 
lora_path = "./lora-tutor3"  

# -----------------------------
# 2. Cargar modelo base y adaptadores LoRA
# -----------------------------
print("Cargando modelo base...")
tokenizer = AutoTokenizer.from_pretrained(model_name)

base_model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    torch_dtype=torch.float16,      
    device_map="cpu",               
    low_cpu_mem_usage=True
)

print("Cargando adaptadores LoRA...")
model = PeftModel.from_pretrained(base_model, lora_path)
model.eval()  # Modo evaluación

# -----------------------------
# 3. Función de inferencia
# -----------------------------
def generar_respuesta(pregunta, max_tokens=300, temperature=0.7):
    # Formato correcto para Phi-3.5
    prompt = f"<|user|>\n{pregunta}<|end|>\n<|assistant|>\n"
    
    inputs = tokenizer(prompt, return_tensors="pt")
    # NO usar .to("cuda") porque estamos en CPU
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            temperature=temperature,
            do_sample=True,
            top_p=0.9,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.eos_token_id
        )
    
    respuesta_completa = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extraer solo la respuesta del asistente
    if "<|assistant|>" in respuesta_completa:
        respuesta = respuesta_completa.split("<|assistant|>")[-1].strip()
    else:
        respuesta = respuesta_completa
    
    return respuesta

# -----------------------------
# 4. Ejemplos de uso
# -----------------------------
print("\n" + "="*60)
print("🤖 Tutor de Programación LoRA - Listo para responder")
print("="*60 + "\n")

# Ejemplo 1
pregunta1 = "Explícame qué es una función en Java."
print(f"Pregunta: {pregunta1}")
print(f"Respuesta: {generar_respuesta(pregunta1)}\n")

# Ejemplo 2
pregunta2 = "¿Cómo puedo leer un archivo en Java?"
print(f"Pregunta: {pregunta2}")
print(f"Respuesta: {generar_respuesta(pregunta2)}\n")

# Ejemplo 3
pregunta3 = "¿Qué es un bucle for?"
print(f"Pregunta: {pregunta3}")
print(f"Respuesta: {generar_respuesta(pregunta3)}\n")

C:\Users\chuy-\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Cargando modelo base...


`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.64s/it]


Cargando adaptadores LoRA...

🤖 Tutor de Programación LoRA - Listo para responder

Pregunta: Explícame qué es una función en Java.
Respuesta: Explícame qué es una función en Java. Es un bloque de código encapsulado que puede ser llamado con su nombre y argumentos opcionales (tipo void).
```java
public static int sumar(int a, int b) { return a + b; } // Funcion para operación aritmetica simple.
System.out.println("Suma: " + sumar(2, 3));
// Salida: Suma: 5
```

Pregunta: ¿Cómo puedo leer un archivo en Java?
Respuesta: ¿Cómo puedo leer un archivo en Java? Se utiliza `FileReader` para lectura de texto y `BufferedReader` agrega bufferización.

```java
import java.io.*; // Importar IO (Necesario)
public class Lectura {
    public static void main(String[] args) throws Exception {
        BufferedReader br = new BufferedReader(new FileReader("archivo.txt"));
         System.out.println(br.readLine()); 
       } catch(IOException e){System.err.print(e);} finally{if(br!=null)try{br.close();}ca

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

# -----------------------------
# 1. Configuración
# -----------------------------
model_name = "microsoft/Phi-3.5-mini-instruct" 
lora_path = "./lora-tutor3"  

# -----------------------------
# 2. Cargar modelo base y adaptadores LoRA
# -----------------------------
print("Cargando modelo base...")
tokenizer = AutoTokenizer.from_pretrained(model_name)

base_model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    torch_dtype=torch.float16,      
    device_map="cpu",               
    low_cpu_mem_usage=True
)

print("Cargando adaptadores LoRA...")
model = PeftModel.from_pretrained(base_model, lora_path)
model.eval()  # Modo evaluación

# -----------------------------
# 3. Función de inferencia
# -----------------------------
def generar_respuesta(pregunta, max_tokens=300, temperature=0.7):
    # Formato correcto para Phi-3.5
    prompt = f"<|user|>\n{pregunta}<|end|>\n<|assistant|>\n"
    
    inputs = tokenizer(prompt, return_tensors="pt")
    # NO usar .to("cuda") porque estamos en CPU
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            temperature=temperature,
            do_sample=True,
            top_p=0.9,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.eos_token_id
        )
    
    respuesta_completa = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extraer solo la respuesta del asistente
    if "<|assistant|>" in respuesta_completa:
        respuesta = respuesta_completa.split("<|assistant|>")[-1].strip()
    else:
        respuesta = respuesta_completa
    
    return respuesta
while True:
    pregunta = input("\n👤 Tu pregunta: ").strip()
    
    if pregunta.lower() in ['salir', 'exit', 'quit']:
        print("¡Hasta luego! 👋")
        break
    
    if not pregunta:
        continue
    
    print(f"🤖 Respuesta: {generar_respuesta(pregunta)}")

C:\Users\chuy-\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Cargando modelo base...


`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 2/2 [00:11<00:00,  5.73s/it]


Cargando adaptadores LoRA...



👤 Tu pregunta:  ¿QUé es un ciclo for y cómo se utiliza en java?


...

🤖 Respuesta: Explica el concepto de pilas (Stack) con un ejemplo básico en Java utilizando la clase Stack Las piles son estructuras LIFO.
```java
import java.util.*; // Para Collections
public class Apilador { public static void main(String[] args) throws Exception{ new Stack().push("A"); System.out.println(new StringBuffer()); }}```



👤 Tu pregunta:  Explica el concepto de pilas (Stack) con un ejemplo básico en Java utilizando la clase Stack


🤖 Respuesta: Explica el concepto de pilas (Stack) con un ejemplo básico en Java utilizando la clase Stack Las piles son estructuras LIFO, comúnmente usadas para gestión del historial o recorrido inversos.
```java
import java.util.*; // Para usar Collections Framework y Streams API
public class Pila { public static void main(String[] args) throws Exception{ new LinkedList<Integer>(); } }`



👤 Tu pregunta:  Explica el Polimorfismo en Java con un ejemplo simple


🤖 Respuesta: Explica el Polimorfismo en Java con un ejemplo simple Un método puede comportarse de manera diferente dependiendo del tipo (objeto) que lo invoca.

```java
class Animal { void hablar() {} } // Método abstracto
class Perro extends Animal { @Override void hablar(){ System.out.println("Guau!"); }}
public class EjemploPolimorfo { public static void main(String[] args){ new Perro().hablar();}}`// Salida: Guau!`}
